<center><strong><font size=+3>Introduction and examples of robust directional and multivariate median estimators</font></center>
<br><br>
</center>
<center><strong><font size=+2>Matyas Molnar and Bojan Nikolic</font><br></strong></center>
<br><center><strong><font size=+1>Astrophysics Group, Cavendish Laboratory, University of Cambridge</font></strong></center>

In [ ]:
import numpy as np
from scipy import stats

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, zoomed_inset_axes
from scipy.stats import shapiro
from scipy.stats.mstats import gmean as geometric_mean

from robstat.robstat import circ_mean_dev, geometric_median, mardia_median, \
mv_median, mv_normality, tukey_median
from robstat.utils import round_down, round_up

In [ ]:
%matplotlib inline

## Directional data

### Mardia Median

Mardia median given by the angle that the circular mean deviation:

$$ d(\tilde\theta) = \pi - \frac{1}{n} \sum_{i=1}^{n} \left| \pi - \left| \theta_i - \tilde\theta \right| \right| $$

where $\tilde\theta$ is the estimate of the preferred direction, and it is used as a measure of dispersion.

The Mardia median occasionally leads to a non-unique estimate of the circular median since there can sometimes be two or more diameters that divide the data equally and have the same circular mean deviation.

Weighted Mardia median:

$$ d(\tilde\theta) = \pi - \frac{1}{\sum \eta_i} \sum_{i=1}^{n} \eta_i \left| \pi - \left| \theta_i - \tilde\theta \right| \right| $$

In [ ]:
# create sample angular data
angle_center = 0.5
np.random.seed(0)
angles = angle_center + np.random.normal(loc=0, scale=0.2, size=200)

x_coords = np.cos(angles)
y_coords = np.sin(angles)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)

circ_rad = 1
lim_rng = circ_rad * 1.25

ax.set(xlim=(-lim_rng, lim_rng), ylim = (-lim_rng, lim_rng))

a_circle = plt.Circle((0, 0), 1, fill=False, color='blue', alpha=0.5)
ax.add_artist(a_circle)

ax.axhline(0, color='grey', ls='--', lw=0.5)
ax.axvline(0, color='grey', ls='--', lw=0.5)
ax.plot(x_coords, y_coords, 'o', color='orange', markersize=4, alpha=0.5)

plt.show()

In [ ]:
# circular mean deviation
np.array([circ_mean_dev(angles, angle_center)]).item()

In [ ]:
# checking uniqueness of Mardia Median

x = mardia_median(angles, init_guess=angle_center-0.3)
y = mardia_median(angles, init_guess=angle_center+0.3)

print('Mardia Medians found:')
print(x)
print(y, '\n')

print('Circular mean: \n{}\n'.format(stats.circmean(angles, nan_policy='omit')))

cmd_x = circ_mean_dev(angles, x).item()
cmd_y = circ_mean_dev(angles, y).item()
print('Circular mean deviations for the Mardia Medians (check):')
print(cmd_x)
print(cmd_y, '\n')

if ~np.isclose(x, y):
    print('The Mardia Median in this example is not unique')
    take_mean = 'Mean'
else:
    take_mean = ''

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=100, subplot_kw={'projection': 'polar'})

ax.set(rlim=(0, lim_rng))
ax.set_rlabel_position(-90)

# ax.plot(np.linspace(0, 2*np.pi, 1000), np.ones(1000), color='blue', linestyle='-', alpha=0.5)
ax.plot(angles, np.ones_like(angles), 'o', color='orange', markersize=4, alpha=0.5, zorder=0)
ax.scatter(np.mean([x, y]), 1, zorder=1, label='{} Mardia median'.format(take_mean))

ax.set_xlim(round_down(angles.min(), 1), round_up(angles.max(), 1))

plt.legend(loc=0)
plt.show()

## Multivariate data

### Geometric Median

The geometric median is defined as the value of the argument $y$ that minimizes the sum of Euclidian distances between $y$ and all points $x_i$:

$$ \underset{y \in \mathbb{R}^d}{\mathrm{arg\,min}} \sum_{i=1}^n || x_i - y ||  $$

Properties & asides:
 - The geometric median has a breakdown point of 0.5: up to half of the sample data may be arbitrarily corrupted, and the median of the samples will still provide a robust estimator for the location of the uncorrupted data
 - The geometric median is unique whenever the points are not collinear
 - Weiszfeld's algorithm for faster computation
 
We can also weight the geometric median (c.f. the Weber problem):

$$ \underset{y \in \mathbb{R}^d}{\mathrm{arg\,min}} \sum_{i=1}^n \eta_i || x_i - y ||  $$

### Tukey Median

Tukey (1975) proposed the halfspace depth as a tool to visually describe bivariate datasets.

For a finite set of data points $X_n = \{x_1, ..., x_n\}$ in $\mathbb{R}^d$, the Tukey depth, or halfspace depth of any point $y \in \mathbb{R}^d$ determines how central the point is inside the data cloud.

The halfspace depth of $y$ is defined as the minimal number of data points in any closed halfspace determined by a hyperplane through $y$:

$$ hdepth(y; X_n) = \underset{|| u || = 1}{\mathrm{min}} \# \{i; u^{\tau} x_i \geq u^{\tau} y \} $$

The set of all points with depth > k is called the kth depth region Dk. The halfspace depth regions form a sequence of nested polyhedra.

In [ ]:
points = np.random.random(500).reshape(-1, 2)*2
points = np.concatenate((points, np.random.random(50).reshape(-1, 2)+5))
points[:, 0] += 2
points[:, 1] += 2
points_c = points[:, 0] + points[:, 1]*1j
sample_gmean = geometric_mean(points_c)
sample_gmed = geometric_median(points_c, weights=None)
sample_tmed = tukey_median(points_c)['barycenter']
bad_med = lambda x : np.nanmedian(x.real) + np.nanmedian(x.imag)*1j
sample_bmed = bad_med(points_c)

In [ ]:
med_ests = list(zip([sample_gmean, sample_gmed, sample_tmed, sample_bmed], \
               ['Geometric Mean', 'Geometric Median', 'Tukey Median', 'Separate Median'], \
               ['ro', 'co', 'yo', 'bo']))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)

ax.scatter(points[:, 0], points[:, 1], alpha=0.5)
for i, med_est in enumerate(med_ests):
    ax.plot(med_est[0].real, med_est[0].imag, med_est[2], label=med_est[1])
ax.set_xlim(1.5, 9)
ax.set_ylim(1.5, 9)

# zoomed in sub region of the original image
axins = zoomed_inset_axes(ax, zoom=6, loc=4)
axins.scatter(points[:, 0], points[:, 1], alpha=0.5)
for i, med_est in enumerate(med_ests):
    axins.plot(med_est[0].real, med_est[0].imag, med_est[2])

x1 = round_down(np.min([med_est[0].real for med_est in med_ests]), 1)
x2 = round_up(np.max([med_est[0].real for med_est in med_ests]), 1)
y1 = round_down(np.min([med_est[0].imag for med_est in med_ests]), 1)
y2 = round_up(np.max([med_est[0].imag for med_est in med_ests]), 1)
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

axins.tick_params(axis='x', direction='in', pad=-15)
mark_inset(ax, axins, loc1=1, loc2=3, fc='none', ec='0.5')

ax.legend(loc='upper left')
plt.show()

In [ ]:
# effect of outliers in data

print('Geometric median')
print(geometric_median(points[:250], weights=None))
print(geometric_median(points, weights=None), '\n')

print('Tukey median')
print(tukey_median(points[:250], weights=None)['barycenter'])
print(tukey_median(points, weights=None)['barycenter'], '\n')

print('Bad median')
print(bad_med(points[:250]))
print(bad_med(points))

### Other location depth notions

 - Simplicial depth
 - Oja depth
 - Projection depth
 - Spatial depth
 
 See e.g. https://www.csun.edu/~ctoth/Handbook/chap58.pdf, https://cran.r-project.org/web/packages/depth/depth.pdf


In [ ]:
for mvm in ['Tukey', 'Oja', 'Liu', 'Spatial', 'CWmed']:
    print('{:7s}: {}'.format(mvm, mv_median(points, method=mvm)))

## Normality tests

### Shapiro-Wilk

The Shapiro–Wilk test (1965) tests the null hypothesis that a sample $x_1, \dots, x_n$ comes from a normal distribution. The test statistic is given by

$$ W = \frac{\left( \sum_{i=1}^n a_i x_{\left( i \right)} \right)^2}{\sum_{i=1}^n \left(x_i - \bar{x} \right)^2} $$

where $x_{\left( i \right)}$ are the ordered sample values (i.e. the the $i$th-smallest number in the sample), $\bar{x}$ is the sample mean, and $a_i$ are constants generated from the means, with $\vec{a} = \left( a_1, \dots, a_n \right)$ given by

$$ \vec{a} = \frac{m^{\intercal}V^{-1}}{C} $$

with vector norm $C$

$$ C = \lVert V^{-1} m \rVert = \left(m^{\intercal} V^{-1} V^{-1} m \right)^{1/2} $$

and $\vec{m} = \left( m_1, \dots, m_n \right)^\intercal$ made of the expected values of the order statistics of *iid* random variables sampled from the standard normal distribution; finally, $V$ is the covariance matrix of those normal order statistics

Small values of $W$ are evidence of departure from normality.

The null-hypothesis of this test is that the sample is normally distributed. Thus, if the $p$ value is less than the chosen threshold level (usually 5%), then the null hypothesis is rejected and there is evidence that the sample data are not normally distributed.

In [ ]:
shapiro_x = shapiro(points[:, 0])
shapiro_y = shapiro(points[:, 1])
print(shapiro_x)
print(shapiro_y)

### Henze-Zirkler

The Henze-Zirkler test (1990) statistic is based on a non-negative functional that measures the distance between two distribution functions, the hypothesized function (which is the multivariate normal) and the observed function.

The HZ test statistic is given by

$$ HZ = \frac{1}{n} \sum^n_{i=1} \sum^n_{j=1} e^{-\frac{\beta^2}{2} D_{ij}} - 2 (1 + \beta^2)^{-\frac{p}{2}} \sum^n_{j=1} e^{-\frac{\beta^2}{2 (1+\beta^2)} D_{i}} + n (1 + 2 \beta^2)^{-\frac{p}{2}} $$

where

$$ p = \# \; \mathrm{variables} $$

$$ \beta = \frac{1}{\sqrt{2}} \left( \frac{n (2p + 1)}{4} \right)^{\frac{1}{p+4}} $$

$$ D_{ij} = (x_i - x_j)' S^{-1} (x_i - x_j) $$

$$ D_{i} = (x_i - \bar{x})' S^{-1} (x_i - \bar{x}) $$

with $D_{i}$ giving the squared Mahalanobis distance of $i^{\mathrm{th}}$ observation to the centroid and $D_{ij}$ the Mahalanobis distance between the $i^{\mathrm{th}}$ and $j^{\mathrm{th}}$ observations, as $S$ is the covariance matrix.

If the data is multivariate normal, $HZ$ is approximately log-normally distributed.

In [ ]:
MVN_res = mv_normality(points)
print(MVN_res)